In [110]:
# data comes from http://web.stanford.edu/~ericyi/project_page/part_annotation/index.html
from pyntcloud import PyntCloud

import sys
import os
if sys.platform == 'darwin':
    data_path = os.getcwd() + "/PartAnnotation"
else:
    data_path = os.getcwd() + "\\PartAnnotation"

In [111]:
data_path

'/Users/shijian/git/3D-CNN/PartAnnotation'

In [112]:
def find_data(data_path):
    data = []
    label = []
    for entry in os.scandir(data_path):
        if entry.is_dir():
            for pts_data in os.scandir(os.path.join(data_path, entry.name, 'points')):
                data.append(os.path.join(data_path, entry.name, 'points', pts_data.name))
                label.append(entry.name)
    return data, label

In [113]:
data, label = find_data(data_path)

In [114]:
my_point_cloud = PyntCloud.from_file(data[9000], sep=" ", header=0, names=["x","y","z"])

In [164]:
my_point_cloud.add_structure("voxelgrid", x_y_z=[32, 32, 32])
my_point_cloud.xyz.shape

(2660, 3)